**Install YOLOv8**

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

**Preparing a custom dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/dataset_cats_dogs.zip

Archive:  /content/drive/MyDrive/dataset_cats_dogs.zip
   creating: dataset_cats_dogs/
   creating: dataset_cats_dogs/cats/
  inflating: dataset_cats_dogs/cats/cat.0.jpg  
  inflating: dataset_cats_dogs/cats/cat.1.jpg  
  inflating: dataset_cats_dogs/cats/cat.2.jpg  
  inflating: dataset_cats_dogs/cats/cat.3.jpg  
  inflating: dataset_cats_dogs/cats/cat.4.jpg  
   creating: dataset_cats_dogs/dogs/
  inflating: dataset_cats_dogs/dogs/dog.0.jpg  
  inflating: dataset_cats_dogs/dogs/dog.1.jpg  
  inflating: dataset_cats_dogs/dogs/dog.2.jpg  
  inflating: dataset_cats_dogs/dogs/dog.3.jpg  
  inflating: dataset_cats_dogs/dogs/dog.4.jpg  


In [ ]:
import os
import cv2
import shutil
import numpy as np
from pathlib import Path
import random

def split(augmentation_directory,to_path):
    main_dir = os.listdir(augmentation_directory)

    os.mkdir(os.path.join(to_path, 'train'))
    os.mkdir(os.path.join(to_path, 'test'))
    os.mkdir(os.path.join(to_path, 'val'))

    for each_dir in main_dir:
        os.mkdir(os.path.join(to_path, 'train', each_dir))
        os.mkdir(os.path.join(to_path, 'test', each_dir))
        os.mkdir(os.path.join(to_path, 'val', each_dir))
        files = os.listdir(os.path.join(augmentation_directory, each_dir))

        train_per = round(len(files) * 0.7)
        valid_per = round(len(files) * 0.2)
        test_per = round(len(files) * 0.1)

        for every_file in files[:train_per]:
            shutil.copyfile(os.path.join(augmentation_directory, each_dir, every_file),
                            os.path.join(to_path, 'train', each_dir, every_file))
        for every_file in files[train_per:train_per + valid_per]:
            shutil.copyfile(os.path.join(augmentation_directory, each_dir, every_file),
                            os.path.join(to_path, 'val', each_dir, every_file))
        for every_file in files[train_per + valid_per:]:
            shutil.copyfile(os.path.join(augmentation_directory, each_dir, every_file),
                            os.path.join(to_path, 'test', each_dir, every_file))


class DataAugmentation:
    """
    Handles with various augmentations for dataset.
    """

    def __init__(self):
        pass

    def fill(self, img, h, w):
        img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
        return img

    def horizontal_shift(self, img, ratio=0.0):
        if ratio > 1 or ratio < 0:
            print('Value should be less than 1 and greater than 0')
            return img
        ratio = random.uniform(-ratio, ratio)
        h, w = img.shape[:2]
        to_shift = w * ratio
        if ratio > 0:
            img = img[:, :int(w - to_shift), :]
        if ratio < 0:
            img = img[:, int(-1 * to_shift):, :]
        img = self.fill(img, h, w)
        return img

    def vertical_shift(self, img, ratio=0.0):
        if ratio > 1 or ratio < 0:
            print('Value should be less than 1 and greater than 0')
            return img
        ratio = random.uniform(-ratio, ratio)
        h, w = img.shape[:2]
        to_shift = h * ratio
        if ratio > 0:
            img = img[:int(h - to_shift), :, :]
        if ratio < 0:
            img = img[int(-1 * to_shift):, :, :]
        img = self.fill(img, h, w)
        return img

    def brightness(self, img, low, high):
        value = random.uniform(low, high)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hsv = np.array(hsv, dtype=np.float64)
        hsv[:, :, 1] = hsv[:, :, 1] * value
        hsv[:, :, 1][hsv[:, :, 1] > 255] = 255
        hsv[:, :, 2] = hsv[:, :, 2] * value
        hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
        hsv = np.array(hsv, dtype=np.uint8)
        img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        return img

    def channel_shift(self, img, value):
        value = int(random.uniform(-value, value))
        img = img + value
        img[:, :, :][img[:, :, :] > 255] = 255
        img[:, :, :][img[:, :, :] < 0] = 0
        img = img.astype(np.uint8)
        return img

    def zoom(self, img, value):
        if value > 1 or value < 0:
            print('Value for zoom should be less than 1 and greater than 0')
            return img
        value = random.uniform(value, 1)
        h, w = img.shape[:2]
        h_taken = int(value * h)
        w_taken = int(value * w)
        h_start = random.randint(0, h - h_taken)
        w_start = random.randint(0, w - w_taken)
        img = img[h_start:h_start + h_taken, w_start:w_start + w_taken, :]
        img = self.fill(img, h, w)
        return img

    def horizontal_flip(self, img, flag):
        if flag:
            return cv2.flip(img, 1)
        else:
            return img

    def vertical_flip(self, img, flag):
        if flag:
            return cv2.flip(img, 0)
        else:
            return img

    def rotation(self, img, angle):
        angle = int(random.uniform(-angle, angle))
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((int(w / 2), int(h / 2)), angle, 1)
        img = cv2.warpAffine(img, M, (w, h))
        return img


    def process(self, dataset_directory, post_process_directory, cl):
        if not os.path.exists(os.path.join(post_process_directory, cl)):
            os.mkdir(os.path.join(post_process_directory, cl))

        for each_file in os.listdir(os.path.join(dataset_directory, cl)):
            filename, file_extension = os.path.splitext(each_file)

            if file_extension in ['.jpg', '.jpeg', '.png']:
                image = cv2.imread(os.path.join(dataset_directory, cl, each_file))
                multi_images = (
                    self.horizontal_shift(image), self.vertical_shift(image), self.brightness(image, 0.5, 3),
                    self.zoom(image, 0.5), self.channel_shift(image, 60), self.horizontal_flip(image, True),
                    self.vertical_flip(image, True), self.rotation(image, 60))

                _file_name = 0
                for each_element in multi_images:
                    image = each_element
                    cv2.imwrite(
                        os.path.join(post_process_directory, cl, f"{each_file[:-4]}" + "_" + f"{_file_name}" + ".jpg"),
                        image)
                    _file_name += 1



def main():
    dataset_directory = Path(r"/content/dataset_cats_dogs")
    augmentation_directory = '/content/augmented_dataset'
    os.mkdir(augmentation_directory)
    target_directory="/content/split_dataset"
    os.mkdir(target_directory)
    cls = ['cats', 'dogs']

    augmentation_obj = DataAugmentation()
    for cl in cls:
        augmentation_obj.process(dataset_directory, augmentation_directory, cl)

    for cl in cls:
        if not os.path.exists(os.path.join(augmentation_directory, cl)):
            os.mkdir(os.path.join(augmentation_directory, cl))

    split(augmentation_directory,target_directory)


if __name__ == "__main__":
    main()

**Custom Training**

In [ ]:
!yolo task=classify mode=train model=yolov8n-cls.pt data=/content/split_dataset epochs=10 imgsz=128

100% 5.28M/5.28M [00:00<00:00, 61.8MB/s]
Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/split_dataset, epochs=1, patience=50, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True,

In [ ]:
!cat /content/runs/classify/train/results.csv | head -10

                  epoch,             train/loss,  metrics/accuracy_top1,  metrics/accuracy_top5,               val/loss,                 lr/pg0,                 lr/pg1,                 lr/pg2
                      1,                0.19301,                 0.4375,                      1,                0.19167,              2.142e-05,              2.142e-05,              2.142e-05


**Inference with Custom Model**

In [ ]:
!yolo task=classify mode=predict model=/content/runs/classify/train/weights/best.pt conf=0.25 source=/content/split_dataset/test/dogs/dog.0_0.jpg

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients

image 1/1 /content/split_dataset/test/dogs/dog.0_0.jpg: 128x128 dogs 0.65, cats 0.35, 42.3ms
Speed: 3.4ms preprocess, 42.3ms inference, 0.2ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs/classify/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
